-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

# Cleaning Data

Most transformations completed with Spark SQL will be familiar to SQL-savvy developers.

As we inspect and clean our data, we'll need to construct various column expressions and queries to express transformations to apply on our dataset.  

Column expressions are constructed from existing columns, operators, and built-in Spark SQL functions. They can be used in **`SELECT`** statements to express transformations that create new columns from datasets. 

Along with **`SELECT`**, many additional query commands can be used to express transformations in Spark SQL, including **`WHERE`**, **`DISTINCT`**, **`ORDER BY`**, **`GROUP BY`**, etc.

In this notebook, we'll review a few concepts that might differ from other systems you're used to, as well as calling out a few useful functions for common operations.

We'll pay special attention to behaviors around **`NULL`** values, as well as formatting strings and datetime fields.

## Learning Objectives
By the end of this lesson, you should be able to:
- Summarize datasets and describe null behaviors
- Retrieve and removing duplicates
- Validate datasets for expected counts, missing values, and duplicate records
- Apply common transformations to clean and transform data

## Run Setup

The setup script will create the data and declare necessary values for the rest of this notebook to execute.

In [0]:
%run ../Includes/Classroom-Setup-4.6

Python interpreter will be restarted.
Python interpreter will be restarted.

Creating the database "dbacademy_manujkumar_joshi_celebaltech_com_dewd_4_6"
Skipping install to "dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/dewd/source/eltwss", dataset already exists

Creating the users_dirty table...(22 seconds / 986 records)

Predefined Paths:
 DA.paths.working_dir: dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/dewd/4.6
 DA.paths.user_db: dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/dewd/4.6/4_6.db
 DA.paths.datasets: dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/dewd/source/eltwss

Predefined tables in dbacademy_manujkumar_joshi_celebaltech_com_dewd_4_6:
 users_dirty

Setup completed in 27 seconds

We'll work with new users records in **`users_dirty`** table for this lesson.

In [0]:
%sql
SELECT * FROM users_dirty

user_id,user_first_touch_timestamp,email,updated
UA000000107338110,1593874163848994,blackburnjohn@gay.biz,2022-07-29T07:11:39.322+0000
UA000000107354520,1593876102569545,reedjennifer@freeman.com,2022-07-29T07:11:39.322+0000
UA000000107359655,1593876696866893,roberthubbard55@hotmail.com,2022-07-29T07:11:39.322+0000
UA000000107362264,1593876996344195,amandafrazier@hotmail.com,2022-07-29T07:11:39.322+0000
UA000000107362440,1593877015690335,laura65@garcia.biz,2022-07-29T07:11:39.322+0000
UA000000107363722,1593877154338249,tmiller67@yahoo.com,2022-07-29T07:11:39.322+0000
UA000000107367850,1593877599590013,xmartinez27@gmail.com,2022-07-29T07:11:39.322+0000
UA000000107370290,1593877869468334,uflynn@anderson.com,2022-07-29T07:11:39.322+0000
UA000000107371743,1593878036347579,nancyellis34@hotmail.com,2022-07-29T07:11:39.322+0000
UA000000107371836,1593878046932741,nporter32@gmail.com,2022-07-29T07:11:39.322+0000


## Inspect Data

Let's start by counting values in each field of our data.

In [0]:
%sql
SELECT count(user_id), count(user_first_touch_timestamp), count(email), count(updated), count(*)
FROM users_dirty

count(user_id),count(user_first_touch_timestamp),count(email),count(updated),count(1)
983,983,138,983,986


Note that **`count(col)`** skips **`NULL`** values when counting specific columns or expressions.

However, **`count(*)`** is a special case that counts the total number of rows (including rows that are only **`NULL`** values).

To count null values, use the **`count_if`** function or **`WHERE`** clause to provide a condition that filters for records where the value **`IS NULL`**.

In [0]:
%sql
SELECT
  count_if(user_id IS NULL) AS missing_user_ids, 
  count_if(user_first_touch_timestamp IS NULL) AS missing_timestamps, 
  count_if(email IS NULL) AS missing_emails,
  count_if(updated IS NULL) AS missing_updates
FROM users_dirty

missing_user_ids,missing_timestamps,missing_emails,missing_updates
3,3,848,3


Clearly there are at least a handful of null values in all of our fields. Let's try to discover what is causing this.

## Distinct Records

Start by looking for distinct rows.

In [0]:
%sql
SELECT count(DISTINCT(*))
FROM users_dirty

"count(DISTINCT user_id, user_first_touch_timestamp, email, updated)"
138


In [0]:
%sql
SELECT count(DISTINCT(user_id))
FROM users_dirty

count(DISTINCT user_id)
917


Because **`user_id`** is generated alongside the **`user_first_touch_timestamp`**, these fields should always be in parity for counts.

In [0]:
%sql
SELECT count(DISTINCT(user_first_touch_timestamp))
FROM users_dirty

count(DISTINCT user_first_touch_timestamp)
917


Here we note that while there are some duplicate records relative to our total row count, we have a much higher number of distinct values.

Let's go ahead and combine our distinct counts with columnar counts to see these values side-by-side.

In [0]:
%sql
SELECT 
  count(user_id) AS total_ids,
  count(DISTINCT user_id) AS unique_ids,
  count(email) AS total_emails,
  count(DISTINCT email) AS unique_emails,
  count(updated) AS total_updates,
  count(DISTINCT(updated)) AS unique_updates,
  count(*) AS total_rows, 
  count(DISTINCT(*)) AS unique_non_null_rows
FROM users_dirty

total_ids,unique_ids,total_emails,unique_emails,total_updates,unique_updates,total_rows,unique_non_null_rows
983,917,138,138,983,1,986,138


Based on the above summary, we know:
* All of our emails are unique
* Our emails contain the largest number of null values
* The **`updated`** column contains only 1 distinct value, but most are non-null

## Deduplicate Rows
Based on the above behavior, what do you expect will happen if we use **`DISTINCT *`** to try to remove duplicate records?

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW users_deduped AS
  SELECT DISTINCT(*) FROM users_dirty;

SELECT * FROM users_deduped

user_id,user_first_touch_timestamp,email,updated
UA000000107391279,1593880174821158,null,2022-07-29T07:11:39.322+0000
UA000000107386838,1593879696869252,null,2022-07-29T07:11:39.322+0000
UA000000107391643,1593880210836589,null,2022-07-29T07:11:39.322+0000
UA000000107400002,1593881092028726,null,2022-07-29T07:11:39.322+0000
UA000000107398145,1593880893591631,null,2022-07-29T07:11:39.322+0000
UA000000107393878,1593880452156698,null,2022-07-29T07:11:39.322+0000
UA000000107399077,1593880993817476,null,2022-07-29T07:11:39.322+0000
UA000000107391995,1593880246854668,null,2022-07-29T07:11:39.322+0000
UA000000107391597,1593880206481325,null,2022-07-29T07:11:39.322+0000
UA000000107382007,1593879169908682,mflores35@gmail.com,2022-07-29T07:11:39.322+0000


Note in the preview above that there appears to be null values, even though our **`COUNT(DISTINCT(*))`** ignored these nulls.

How many rows do you expect passed through this **`DISTINCT`** command?

In [0]:
%sql
SELECT COUNT(*) FROM users_deduped

count(1)
984


Note that we now have a completely new number.

Spark skips null values while counting values in a column or counting distinct values for a field, but does not omit rows with nulls from a **`DISTINCT`** query.

Indeed, the reason we're seeing a new number that is 1 higher than previous counts is because we have 3 rows that are all nulls (here included as a single distinct row).

In [0]:
%sql
SELECT * FROM users_dirty
WHERE
  user_id IS NULL AND
  user_first_touch_timestamp IS NULL AND
  email IS NULL AND
  updated IS NULL

user_id,user_first_touch_timestamp,email,updated
null,null,null,null
null,null,null,null
null,null,null,null


## Deduplicate Based on Specific Columns

Recall that **`user_id`** and **`user_first_touch_timestamp`** should form unique tuples, as they are both generated when a given user is first encountered.

We can see that we have some null values in each of these fields; exclude nulls counting the distinct number of pairs for these fields will get us the correct count for distinct values in our table.

In [0]:
%sql
SELECT COUNT(DISTINCT(user_id, user_first_touch_timestamp))
FROM users_dirty
WHERE user_id IS NOT NULL

"count(DISTINCT named_struct(user_id, user_id, user_first_touch_timestamp, user_first_touch_timestamp))"
917


Here, we'll use these distinct pairs to remove unwanted rows from our data.

The code below uses **`GROUP BY`** to remove duplicate records based on **`user_id`** and **`user_first_touch_timestamp`**.

The **`max()`** aggregate function is used on the **`email`** column as a hack to capture non-null emails when multiple records are present; in this batch, all **`updated`** values were equivalent, but we need to use an aggregate function to keep this value in the result of our group by.

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW deduped_users AS
SELECT user_id, user_first_touch_timestamp, max(email) AS email, max(updated) AS updated
FROM users_dirty
WHERE user_id IS NOT NULL
GROUP BY user_id, user_first_touch_timestamp;

SELECT count(*) FROM deduped_users

count(1)
917


## Validate Datasets
We've visually confirmed that our counts are as expected, based our manual review.
 
Below, we programmatically do some validation using simple filters and **`WHERE`** clauses.

Validate that the **`user_id`** for each row is unique.

In [0]:
%sql
SELECT max(row_count) <= 1 no_duplicate_ids FROM (
  SELECT user_id, count(*) AS row_count
  FROM deduped_users
  GROUP BY user_id)

no_duplicate_ids
true


Confirm that each email is associated with at most one **`user_id`**.

In [0]:
%sql
SELECT max(user_id_count) <= 1 at_most_one_id FROM (
  SELECT email, count(user_id) AS user_id_count
  FROM deduped_users
  WHERE email IS NOT NULL
  GROUP BY email)

at_most_one_id
true


## Date Format and Regex
Now that we've removed null fields and eliminated duplicates, we may wish to extract further value out of the data.

The code below:
- Correctly scales and casts the **`user_first_touch_timestamp`** to a valid timestamp
- Extracts the calendar data and clock time for this timestamp in human readable format
- Uses **`regexp_extract`** to extract the domains from the email column using regex

In [0]:
%sql
SELECT *,
  date_format(first_touch, "MMM d, yyyy") AS first_touch_date,
  date_format(first_touch, "HH:mm:ss") AS first_touch_time,
  regexp_extract(email, "(?<=@).+", 0) AS email_domain
FROM (
  SELECT *,
    CAST(user_first_touch_timestamp / 1e6 AS timestamp) AS first_touch 
  FROM deduped_users
)

user_id,user_first_touch_timestamp,email,updated,first_touch,first_touch_date,first_touch_time,email_domain
UA000000107335605,1593873851950592,danielcarrillo@smith.com,2022-07-29T07:11:39.322+0000,2020-07-04T14:44:11.950+0000,"Jul 4, 2020",14:44:11,smith.com
UA000000107338110,1593874163848994,blackburnjohn@gay.biz,2022-07-29T07:11:39.322+0000,2020-07-04T14:49:23.848+0000,"Jul 4, 2020",14:49:23,gay.biz
UA000000107342625,1593874719224880,xjoseph@miller.biz,2022-07-29T07:11:39.322+0000,2020-07-04T14:58:39.224+0000,"Jul 4, 2020",14:58:39,miller.biz
UA000000107345540,1593875054653990,theresahuber@yahoo.com,2022-07-29T07:11:39.322+0000,2020-07-04T15:04:14.653+0000,"Jul 4, 2020",15:04:14,yahoo.com
UA000000107347151,1593875250452082,melissamcdaniel@gmail.com,2022-07-29T07:11:39.322+0000,2020-07-04T15:07:30.452+0000,"Jul 4, 2020",15:07:30,gmail.com
UA000000107349198,1593875490748942,hwhite@hall.com,2022-07-29T07:11:39.322+0000,2020-07-04T15:11:30.748+0000,"Jul 4, 2020",15:11:30,hall.com
UA000000107351411,1593875742437934,castrodavid@valdez.com,2022-07-29T07:11:39.322+0000,2020-07-04T15:15:42.437+0000,"Jul 4, 2020",15:15:42,valdez.com
UA000000107354520,1593876102569545,reedjennifer@freeman.com,2022-07-29T07:11:39.322+0000,2020-07-04T15:21:42.569+0000,"Jul 4, 2020",15:21:42,freeman.com
UA000000107355987,1593876269167805,robert03@cohen-kaiser.net,2022-07-29T07:11:39.322+0000,2020-07-04T15:24:29.167+0000,"Jul 4, 2020",15:24:29,cohen-kaiser.net
UA000000107356825,1593876365676183,sullivanjohn@dunn.com,2022-07-29T07:11:39.322+0000,2020-07-04T15:26:05.676+0000,"Jul 4, 2020",15:26:05,dunn.com


Run the following cell to delete the tables and files associated with this lesson.

In [0]:
%python
DA.cleanup()

Dropping the database "dbacademy_manujkumar_joshi_celebaltech_com_dewd_4_6"
Removing the working directory "dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/dewd/4.6"

-sandbox
&copy; 2022 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="https://help.databricks.com/">Support</a>